In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/A"

/content/drive/MyDrive/A


In [3]:
%%capture
!pip3 install tensorflow
!pip3 install nibabel
!pip3 install scipy
!pip3 install -r requirements.txt
!pip3 install torchio

In [4]:
import os
import time
import zipfile
import numpy as np
import nibabel as nib
from tensorflow import keras
from scipy import ndimage

from torch.optim import Adam
from torch import nn
import generate_model
import models.EfficientNet
import torch
from torch.utils.data import DataLoader
import torchio as tio

%matplotlib notebook
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Download url of normal CT scans and abnormal scans.

if not os.path.exists("CT-0.zip"):
  url = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-0.zip"
  filename = os.path.join(os.getcwd(), "CT-0.zip")
  keras.utils.get_file(filename, url)


if not os.path.exists("CT-23.zip"):
  url = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-23.zip"
  filename = os.path.join(os.getcwd(), "CT-23.zip")
  keras.utils.get_file(filename, url)


if not os.path.exists("./CTData"):
    os.makedirs("CTData")

    with zipfile.ZipFile("CT-0.zip", "r") as z_fp:
        z_fp.extractall("./CTData/")

    with zipfile.ZipFile("CT-23.zip", "r") as z_fp:
        z_fp.extractall("./CTData/")


normal_scan_paths = [
    (os.path.join(os.getcwd(), "CTData/CT-0", x), 0)
    for x in os.listdir("CTData/CT-0")
]

abnormal_scan_paths = [
    (os.path.join(os.getcwd(), "CTData/CT-23", x), 1)
    for x in os.listdir("CTData/CT-23")
]

print("CT scans with normal lung tissue: " + str(len(normal_scan_paths)))
print("CT scans with abnormal lung tissue: " + str(len(abnormal_scan_paths)))

CT scans with normal lung tissue: 100
CT scans with abnormal lung tissue: 100


In [6]:
# both dataset should have equal samples.

split = 0.7

permutation = np.random.permutation(range(len(normal_scan_paths)))
train = permutation[:int(len(normal_scan_paths) * split)]
val = permutation[int(len(normal_scan_paths) * split):]

train_subjects = []
for i in train:
  subject_path_n, label_n = normal_scan_paths[i]
  subject_path_a, label_a = abnormal_scan_paths[i]

  subject_n = tio.Subject(t1 = tio.ScalarImage(subject_path_n), label = label_n)
  subject_a = tio.Subject(t1 = tio.ScalarImage(subject_path_a), label = label_a)

  train_subjects.append(subject_n)
  train_subjects.append(subject_a)



val_subjects = []
for i in val:
  subject_path_n, label_n = normal_scan_paths[i]
  subject_path_a, label_a = abnormal_scan_paths[i]

  subject_n = tio.Subject(t1 = tio.ScalarImage(subject_path_n), label = label_n)
  subject_a = tio.Subject(t1 = tio.ScalarImage(subject_path_a), label = label_a)

  val_subjects.append(subject_n)
  val_subjects.append(subject_a)

In [7]:
normalize = tio.RescaleIntensity(out_min_max = (0, 1), in_min_max = (-1000, 400))
resize = tio.Resize((128, 128, 64))
affine = tio.OneOf({tio.RandomAffine(): 0.8 })

train_transform = tio.Compose([normalize, resize, affine])
val_transform = tio.Compose([normalize, resize])

train_subjects_dataset = tio.SubjectsDataset(train_subjects, transform=train_transform)
val_subjects_dataset = tio.SubjectsDataset(val_subjects, transform=val_transform)

training_loader = DataLoader(train_subjects_dataset, shuffle=True, batch_size=4)
validation_loader = DataLoader(val_subjects_dataset, batch_size=4)

train_steps = len(training_loader.dataset) // 4
val_steps = len(validation_loader.dataset) // 4

In [8]:
def train_model(epoch, model, device, lossFn, optimizer, history, mute = True):
	print("[INFO] training the network...")
	startTime = time.time()
	history["train_loss"] = []
	history["train_acc"] = []
	history["val_loss"] = []
	history["val_acc"] = []
	for e in range(0, epoch):
		model.train()
		totalTrainLoss = 0
		totalValLoss = 0
		trainCorrect = 0
		valCorrect = 0
		for subjects_batch in training_loader:
			x = subjects_batch['t1'][tio.DATA].to(device)
			y = subjects_batch['label'].to(device)
			optimizer.zero_grad()
			pred = model(x)
			loss = lossFn(pred, y)
			loss.backward()
			optimizer.step()
			totalTrainLoss += loss
			trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

		with torch.no_grad():
			model.eval()
			for subjects_batch in validation_loader:
				x = subjects_batch['t1'][tio.DATA].to(device)
				y = subjects_batch['label'].to(device)
				pred = model(x)
				totalValLoss += lossFn(pred, y)
				valCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

		avgTrainLoss = totalTrainLoss / train_steps
		avgValLoss = totalValLoss / val_steps
		trainCorrect = trainCorrect / len(training_loader.dataset)
		valCorrect = valCorrect / len(validation_loader.dataset)
		history["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
		history["train_acc"].append(trainCorrect)
		history["val_loss"].append(avgValLoss.cpu().detach().numpy())
		history["val_acc"].append(valCorrect)
		if not mute:
			print("[INFO] EPOCH: {}/{}".format(e + 1, epoch))
			print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
				avgTrainLoss, trainCorrect))
			print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
				avgValLoss, valCorrect))
	print("[INFO] total time taken to train the model: {}s".format(
		round(time.time() - startTime, 2)))

def plot_history(history):
	fig, ax = plt.subplots(1, 2, figsize=(20, 3))
	ax = ax.ravel()
	for i, metric in enumerate(["acc", "loss"]):
			ax[i].plot(history["train_" + metric])
			ax[i].plot(history["val_" + metric])
			ax[i].set_title("Model {}".format(metric))
			ax[i].set_xlabel("epochs")
			ax[i].set_ylabel(metric)
			ax[i].legend(["train", "val"])

In [ ]:
# EfficientNet b0
model = models.EfficientNet.EfficientNet3D.from_name(
            'efficientnet-b0', 
            override_params={'num_classes': 2}, 
            in_channels=1).cuda()
lossFn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 10e-4, weight_decay = 10e-5)
history_0 = {}

train_model(200, model, device, lossFn, optimizer, history_0, mute = False)
plot_history(history_0)

[INFO] training the network...
[INFO] EPOCH: 1/200
Train loss: 0.875454, Train accuracy: 0.5071
Val loss: 0.693485, Val accuracy: 0.5000

[INFO] EPOCH: 2/200
Train loss: 0.800749, Train accuracy: 0.4214
Val loss: 0.694800, Val accuracy: 0.5000

[INFO] EPOCH: 3/200
Train loss: 0.727618, Train accuracy: 0.5429
Val loss: 0.693163, Val accuracy: 0.5000

[INFO] EPOCH: 4/200
Train loss: 0.760523, Train accuracy: 0.4071
Val loss: 0.699752, Val accuracy: 0.5000

[INFO] EPOCH: 5/200
Train loss: 0.736830, Train accuracy: 0.4714
Val loss: 0.693147, Val accuracy: 0.5000

[INFO] EPOCH: 6/200
Train loss: 0.756135, Train accuracy: 0.4429
Val loss: 0.705288, Val accuracy: 0.5000

[INFO] EPOCH: 7/200
Train loss: 0.757724, Train accuracy: 0.4714
Val loss: 0.693170, Val accuracy: 0.5000

[INFO] EPOCH: 8/200
Train loss: 0.732631, Train accuracy: 0.5429
Val loss: 0.716309, Val accuracy: 0.5000

[INFO] EPOCH: 9/200
Train loss: 0.789131, Train accuracy: 0.4429
Val loss: 0.693478, Val accuracy: 0.5000

[INFO]

In [11]:
# EfficientNet b0
model = models.EfficientNet.EfficientNet3D.from_name(
            'efficientnet-b0', 
            override_params={'num_classes': 2}, 
            in_channels=1).cuda()
lossFn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 10e-4, weight_decay = 5 * 10e-3)
history_1 = {}

train_model(200, model, device, lossFn, optimizer, history_1, mute = False)
plot_history(history_1)

[INFO] training the network...
[INFO] EPOCH: 1/200
Train loss: 0.948143, Train accuracy: 0.4500
Val loss: 0.694396, Val accuracy: 0.5000

[INFO] EPOCH: 2/200
Train loss: 0.776182, Train accuracy: 0.4714
Val loss: 0.693220, Val accuracy: 0.5000

[INFO] EPOCH: 3/200
Train loss: 0.720956, Train accuracy: 0.4929
Val loss: 0.693436, Val accuracy: 0.5000

[INFO] EPOCH: 4/200
Train loss: 0.732843, Train accuracy: 0.5143
Val loss: 0.693369, Val accuracy: 0.5000

[INFO] EPOCH: 5/200
Train loss: 0.729529, Train accuracy: 0.5000
Val loss: 0.693282, Val accuracy: 0.5000

[INFO] EPOCH: 6/200
Train loss: 0.728729, Train accuracy: 0.4929
Val loss: 0.693877, Val accuracy: 0.5000

[INFO] EPOCH: 7/200
Train loss: 0.710280, Train accuracy: 0.5071
Val loss: 0.693389, Val accuracy: 0.5000

[INFO] EPOCH: 8/200
Train loss: 0.705760, Train accuracy: 0.5071
Val loss: 0.693227, Val accuracy: 0.5000

[INFO] EPOCH: 9/200
Train loss: 0.715116, Train accuracy: 0.4286
Val loss: 0.693162, Val accuracy: 0.5000

[INFO]

KeyboardInterrupt: ignored

In [9]:
# EfficientNet b0
model = models.EfficientNet.EfficientNet3D.from_name(
            'efficientnet-b0', 
            override_params={'num_classes': 2}, 
            in_channels=1).cuda()
lossFn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 10e-4, weight_decay = 5 * 10e-3)
history_2 = {}

train_model(200, model, device, lossFn, optimizer, history_2, mute = False)
plot_history(history_2)

[INFO] training the network...
[INFO] EPOCH: 1/200
Train loss: 0.843385, Train accuracy: 0.5357
Val loss: 0.693164, Val accuracy: 0.5000

[INFO] EPOCH: 2/200
Train loss: 0.858637, Train accuracy: 0.4571
Val loss: 0.693329, Val accuracy: 0.5000

[INFO] EPOCH: 3/200
Train loss: 0.754451, Train accuracy: 0.5214
Val loss: 0.693239, Val accuracy: 0.5000

[INFO] EPOCH: 4/200
Train loss: 0.732336, Train accuracy: 0.4714
Val loss: 0.693155, Val accuracy: 0.5000

[INFO] EPOCH: 5/200
Train loss: 0.725894, Train accuracy: 0.4786
Val loss: 0.693831, Val accuracy: 0.5000

[INFO] EPOCH: 6/200
Train loss: 0.713575, Train accuracy: 0.4714
Val loss: 0.693174, Val accuracy: 0.5000

[INFO] EPOCH: 7/200
Train loss: 0.727987, Train accuracy: 0.4500
Val loss: 0.693327, Val accuracy: 0.5000

[INFO] EPOCH: 8/200
Train loss: 0.708684, Train accuracy: 0.5071
Val loss: 0.693157, Val accuracy: 0.5000

[INFO] EPOCH: 9/200
Train loss: 0.729885, Train accuracy: 0.4214
Val loss: 0.693198, Val accuracy: 0.5000

[INFO]

<IPython.core.display.Javascript object>

In [12]:
def plot_history(history):
	fig, ax = plt.subplots(1, 2, figsize=(20, 3))
	ax = ax.ravel()
	for i, metric in enumerate(["acc", "loss"]):
			ax[i].plot(history["train_" + metric])
			ax[i].plot(history["val_" + metric])
			ax[i].set_title("Model {}".format(metric))
			ax[i].set_xlabel("epochs")
			ax[i].set_ylabel(metric)
			ax[i].legend(["train", "val"])
   
plot_history(history_2)

<IPython.core.display.Javascript object>

In [ ]:
# !jupyter nbconvert --to script ct.ipynb